In [35]:
import numpy as np
from collections import defaultdict, deque
from queue import PriorityQueue

In [36]:
nodes = 8
edges = 11
MaxDepth = nodes

In [37]:
class Graph:
    def __init__(self, nodes, edges):
        self.nodes = nodes
        self.edges = edges
        # self.graph = {}
        self.graph = defaultdict(dict)
        self.heuristic = defaultdict(int)
    def set_heuristic(self, x, h):
        self.heuristic[x] = h
    def add_edge_dirless(self, x, y, w):
        self.graph[x][y] = w
        self.graph[y][x] = w
    def visit_whole_graph(self):
        for v, v_items in self.graph.items():
            for u, w in v_items.items():
                print("{} to {} costs {}".format(v, u, w))
            

In [38]:
G = Graph(nodes, edges)

G.add_edge_dirless('A', 'B', 3)
G.add_edge_dirless('A', 'H', 4)
G.add_edge_dirless('C', 'B', 4)
G.add_edge_dirless('H', 'B', 5)
G.add_edge_dirless('C', 'G', 3)
G.add_edge_dirless('C', 'D', 8)
G.add_edge_dirless('G', 'H', 2)
G.add_edge_dirless('G', 'D', 8)
G.add_edge_dirless('G', 'F', 4)
G.add_edge_dirless('D', 'E', 2)
G.add_edge_dirless('D', 'F', 3)

G.set_heuristic('A', 15)
G.set_heuristic('B', 14)
G.set_heuristic('C', 10)
G.set_heuristic('D', 2)
G.set_heuristic('E', 0)
G.set_heuristic('F', 5)
G.set_heuristic('G', 9)
G.set_heuristic('H', 11)

In [39]:
G.visit_whole_graph()
# print(G.graph)

A to B costs 3
A to H costs 4
B to A costs 3
B to C costs 4
B to H costs 5
H to A costs 4
H to B costs 5
H to G costs 2
C to B costs 4
C to G costs 3
C to D costs 8
G to C costs 3
G to H costs 2
G to D costs 8
G to F costs 4
D to C costs 8
D to G costs 8
D to E costs 2
D to F costs 3
F to G costs 4
F to D costs 3
E to D costs 2


In [40]:
DFS_File = open('./DFS_Expansion.txt',"w")
node_min_cost = defaultdict(lambda: 0x7fffffff)
route = []
MAXCOST = 0x7fffffff
def DFS_for_val(G:Graph, source:str, dest:str, cost:int, depth:int) -> int:
    """深度优先搜索图

    搜索树限制高度为节点数
    """
    route.append(source)
    DFS_File.write("{: <30} Current cost: {}\n".format("-".join(route), cost))

    node_min_cost[source] = min(node_min_cost[source], cost)

    if depth > G.nodes:
        route.pop()
        return MAXCOST
    
    if source == dest:
        route.pop()
        return cost
    
    # init
    min_cost = MAXCOST
    for u, w in G.graph[source].items():
        min_cost = min(min_cost, DFS_for_val(G, u, dest, cost+w, depth+1))

    route.pop()
    return min_cost

shortest_route = []
def DFS_for_route(G:Graph, source:str, dest:str):
    if source == dest:
        print("最短路径："+'-'.join(shortest_route))
        return
    for u, w in G.graph[source].items():
        if u not in shortest_route and node_min_cost[source] + w == node_min_cost[u]:
            shortest_route.append(u)
            DFS_for_route(G, u, dest)
            shortest_route.pop()

In [41]:
print(DFS_for_val(G, 'A', 'E', 0, 0))
shortest_route.append('A')
DFS_for_route(G, 'A', 'E')
DFS_File.close()

15
最短路径：A-H-G-F-D-E


In [42]:
BFS_File = open("./BFS_Expansion.txt",'w')
class node:
    def __init__(self, source:str, depth, cost:int, Astar_val, route:list):
        self.depth = depth
        self.source = source
        self.cost = cost
        self.Astar_val = Astar_val
        route.append(source)
        self.route = route.copy()
    def __lt__(self, other):
        return self.Astar_val < other.Astar_val
queue = deque()

def BFS(G:Graph, source:str, dest:str):
    min_cost = MAXCOST
    queue.append(node(source, 0, 0, 0, []))
    while len(queue) != 0:
        try:
            source_node = queue.popleft()
            BFS_File.write("{: <30} Current cost: {}\n".format("-".join(source_node.route), source_node.cost))
        except IndexError:
            print("[ERROR] 队空了！")
            return
        
        if source_node.depth > G.nodes:
            continue
        if source_node.source == dest:
            min_cost = min(min_cost,source_node.cost)
            continue

        for next_node, weight in G.graph[source_node.source].items():
            queue.append(node(next_node, source_node.depth+1, source_node.cost +
                         weight, source_node.cost+weight, source_node.route.copy()))
    return min_cost


In [43]:
BFS_min_cost = BFS(G, 'A', 'E')
print(BFS_min_cost)
BFS_File.close()

15


In [44]:
queue = PriorityQueue()
UCS_File = open("./UCS_Expansion.txt",'w')

def UCS(G:Graph, source:str, dest:str):
    min_cost = MAXCOST
    queue.put(node(source, 0, 0, 0, []))
    while not queue.empty():
        try:
            source_node = queue.get()
            UCS_File.write("{: <30} Current cost: {}\n".format("-".join(source_node.route), source_node.cost))
        except IndexError:
            print("[ERROR] 队空了！")
            return
        
        if source_node.depth > G.nodes:
            continue
        if source_node.source == dest:
            min_cost = min(min_cost,source_node.cost)
            UCS_File.close()
            return min_cost

        for next_node, weight in G.graph[source_node.source].items():
            queue.put(node(next_node, source_node.depth+1, source_node.cost +
                         weight, source_node.cost+weight, source_node.route.copy()))
    return min_cost

In [45]:
print(UCS(G, 'A', 'E'))

15


In [46]:
Greedy_File = open("./Greedy_Expansion.txt", "w")
def Greedy(G:Graph, source:str, dest:str, cost:int, path:list)->int:
    min_heuristic_val = MAXCOST
    min_heuristic_node = None
    path.append(source)

    if source == dest:
        Greedy_File.write('-'.join(path)+'\n')
        return cost

    for next_node, _weight in G.graph[source].items():
        heuristic = G.heuristic[next_node]
        if heuristic < min_heuristic_val:
            min_heuristic_val = heuristic
            min_heuristic_node = next_node
    return Greedy(G, min_heuristic_node, dest, cost+G.graph[source][min_heuristic_node], path)

In [47]:
print(Greedy(G, 'A', 'E', 0, []))
Greedy_File.close()

16


In [50]:
Astar_File = open("./Astar_Expansion.txt","w")
queue = PriorityQueue()

def Astar(G:Graph, source:str, dest:str):
    min_cost = MAXCOST
    queue.put(node(source, 0, 0, 0, []))
    while not queue.empty():
        try:
            source_node = queue.get()
            Astar_File.write("{: <30} Current cost: {}\n".format("-".join(source_node.route), source_node.cost))
        except IndexError:
            print("[ERROR] 队空了！")
            return
        
        if source_node.depth > G.nodes:
            continue
        if source_node.source == dest:
            min_cost = min(min_cost,source_node.cost)
            Astar_File.close()
            return min_cost

        for next_node, weight in G.graph[source_node.source].items():
            queue.put(node(next_node, source_node.depth+1, source_node.cost +
                         weight, source_node.cost+weight+G.heuristic[next_node], source_node.route.copy()))
    return min_cost

In [51]:
print(Astar(G, 'A', 'E'))
Astar_File.close()

15
